In [1]:
from datetime import datetime, timedelta, timezone
import pandas as pd
from robo_trader import Portfolio, Interval, DummyExchange
from robo_trader.strategies import LorentzianStrategy, LorentzianSettings, LorentzianOptimizer, LorentzianOptimizerSettings
from robo_trader.price_providers import YFinancePriceProvider
import json

In [2]:
# SYMBOL = "BTC-GBP"
# inception_date = datetime(2000, 1, 1, tzinfo=timezone.utc)
# train_since = datetime(2016, 7, 1, tzinfo=timezone.utc)
# start_date = datetime(2018, 7, 1, tzinfo=timezone.utc)
# end_date = datetime(2019, 7, 1, tzinfo=timezone.utc)
# interval = Interval.ONE_DAY
# period = 28

SYMBOL = "BTC-GBP"
inception_date = datetime(2000, 1, 1, tzinfo=timezone.utc)
train_since = datetime(2024, 3, 11, tzinfo=timezone.utc)
start_date = datetime(2024, 4, 11, tzinfo=timezone.utc)
end_date = datetime(2024, 5, 11, tzinfo=timezone.utc)
interval = Interval.ONE_HOUR
period = 24

price_provider = YFinancePriceProvider()
exchange = DummyExchange({SYMBOL: 100}, 0.004)
portfolio = Portfolio(SYMBOL, exchange, inception_date = inception_date, cash_reserve_percentage = 1)

In [3]:
default_settings = LorentzianSettings(
    use_RSI=True,
    use_WT=True,
    use_CCI=True,
    use_ADX=False,
    use_MFI=True,
    useKernelSmoothing=True,
    useRegimeFilter=True,
    useAdxFilter=False,
    useVolatilityFilter=False,
    useEmaFilter=False,
    useSmaFilter=False,
    useDynamicExits=True,

    neighborsCount=8, maxBarsBack=720,
    emaPeriod=200, smaPeriod=200,
    lookbackWindow=16, relativeWeight=8.0, regressionLevel=18, crossoverLag=0,
    regimeThreshold=1, adxThreshold=20,
    RSI_param1=24, RSI_param2=5,
    WT_param1=6, WT_param2=11,
    CCI_param1=24, CCI_param2=6,
    ADX_param1=21, ADX_param2=2,
    MFI_param1=14
)

optimizer_settings = LorentzianOptimizerSettings(

)

settings = LorentzianOptimizer.get_optimal_settings(
    SYMBOL, price_provider, interval, portfolio, optimizer_settings, default_settings, start_date, end_date, period, train_since
)

# Write settings to disk
settings_file_path = "optimal_settings.json"
with open(settings_file_path, "w") as settings_file:
    settings_dict = settings.__dict__
    json.dump(settings_dict, settings_file, indent=4)
    
print(f"Optimal settings have been written to {settings_file_path}")

Optimal settings have been written to optimal_settings.json


In [4]:
with open(settings_file_path, "r") as settings_file:
    optimal_settings_data = json.load(settings_file)
    optimal_settings = LorentzianSettings(**optimal_settings_data)

strategy = LorentzianStrategy(SYMBOL, price_provider, interval, portfolio, optimal_settings)

perf = strategy.backtest(start_date=start_date, end_date=end_date, period=period, train_since=train_since)


In [5]:
# Set display options
pd.set_option('display.max_rows', None)  # Adjust None to your specific needs
pd.set_option('display.max_columns', None)  # Adjust None as needed
pd.set_option('display.width', 1000)  # Adjust the width to fit your notebook

date_format = lambda x: x.strftime('%Y-%m-%d %H:%M')
perf = perf.style.format({'start_date': date_format, 'end_date': date_format})

display(perf)

,start_date,end_date,price_start,price_end,valuation_start,valuation_end,asset_return,portfolio_return
0,2024-04-11 00:00,2024-04-11 00:00,56343.437500,56244.683594,1000.000000,1000.000000,-0.001753,0.000000
1,2024-04-11 00:00,2024-04-12 00:00,56244.683594,56004.929688,1000.000000,1000.000000,-0.004263,0.000000
2,2024-04-12 00:00,2024-04-13 00:00,56004.929688,53707.167969,1000.000000,1000.000000,-0.041028,0.000000
3,2024-04-13 00:00,2024-04-14 00:00,53707.167969,51107.062500,1000.000000,1000.000000,-0.048413,0.000000
4,2024-04-14 00:00,2024-04-15 00:00,51107.062500,52612.289062,1000.000000,1000.000000,0.029452,0.000000
5,2024-04-15 00:00,2024-04-16 00:00,52612.289062,50873.664062,1000.000000,982.748248,-0.033046,-0.017252
6,2024-04-16 00:00,2024-04-17 00:00,50873.664062,51509.640625,982.748248,982.872820,0.012501,0.000127
7,2024-04-17 00:00,2024-04-18 00:00,51509.640625,49380.281250,982.872820,982.455732,-0.041339,-0.000424
8,2024-04-18 00:00,2024-04-19 00:00,49380.281250,50533.406250,982.455732,987.980723,0.023352,0.005624
9,2024-04-19 00:00,2024-04-20 00:00,50533.406250,51407.269531,987.980723,988.218584,0.017293,0.000241
